In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType,TimestampType
from pyspark.sql.types import DateType
from pyspark.sql.functions import *

#pip install findspark
#/usr/local/opt/apache-spark/libexec/sbin
#./start-thriftserver.sh

In [2]:
import findspark
findspark.init()

In [5]:
spark = SparkSession\
    .builder\
    .appName("spark")\
    .config('spark.driver.memory', '32g')\
    .config("hive.server2.thrift.port", 10000)\
    .config("spark.sql.hive.thriftServer.singleSession", True)\
    .enableHiveSupport()\
    .getOrCreate()


In [6]:
schema = StructType([
    StructField("ID", IntegerType()),
    StructField("Speed", DoubleType()),
    StructField("TravelTime", IntegerType()),
    StructField("Status", IntegerType()),
    StructField("timedate", StringType()),
    StructField("LinkId", IntegerType()),
    StructField("LinkPoints", StringType()),
    StructField("EncodedLinkPoints", StringType()),
    StructField("EncodedPolyLineLvls", StringType()),
    StructField("Owner", StringType()),
    StructField("TranscomId", IntegerType()),
    StructField("Borough", StringType()),
    StructField("Link_Name", StringType())
])

schema1 = "ID bigint, Speed float, TravelTime int, Status int, TimeDate string, LinkId int, LinkPoints string, EncodedLinkPoints string, EncodedPolyLineLvls string, Owner string, TranscomId int, Borough string,Link_Name string"
df2 = spark.read.format("csv").option("header", "true").schema(schema).load("file:/Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv")
df2 = df2.withColumn("timedate", to_timestamp("timedate", 'MM/dd/yyyy hh:mm:ss a'))
df2 = df2.na.drop()
df2 = df2.drop(*("Status","EncodedLinkPoints","EncodedPolyLineLvls","TranscomId"))
df2.printSchema()
df2.createOrReplaceTempView("dataset")
# .option("inferSchema", "true")

root
 |-- ID: integer (nullable = true)
 |-- Speed: double (nullable = true)
 |-- TravelTime: integer (nullable = true)
 |-- timedate: timestamp (nullable = true)
 |-- LinkId: integer (nullable = true)
 |-- LinkPoints: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)



In [ ]:
data2022 = spark.sql("select id, to_json(split(LinkPoints,' ')) as `Link Points`, count(LinkPoints) from dataset group by id,`Link Points` order by id asc")
data2022.show()

In [44]:
data2022.select(col('id'),exp(split(col('Link Points'),' ')))

22/03/23 22:48:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


+---+--------------------+
| id|                 col|
+---+--------------------+
|  1|["40.74047,-74.00...|
|  2|["40.73933,-74.01...|
|  3|["40.76375,-73.99...|
|  4|["40.7607,-74.002...|
|106|["40.77158,-73.99...|
|107|["40.64289,-74.21...|
|108|["40.62464,-74.17...|
|110|["40.5256,-74.230...|
|119|["40.70631,-74.01...|
|122|["40.7073904,-74....|
|123|["40.70738,-74.01...|
|124|["40.68036,-74.00...|
|126|["40.8271606,-73....|
|126|["40.8271606,-73....|
|129|["40.8240706,-73....|
|137|["40.8242005,-73....|
|140|["40.79789,-73.91...|
|141|["40.772251,-73.9...|
|142|["40.83037,-73.85...|
|145|["40.7081105,-73....|
+---+--------------------+
only showing top 20 rows



In [39]:
data2022.write.format('json').mode('overwrite').save('file:/Users/minoseah629/Desktop/ait614/linkpoints by id.json')


22/03/23 22:35:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


In [6]:
data2022.count()

22/03/22 23:33:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


1893262

In [1]:
spark.stop()

NameError: name 'spark' is not defined